ChIP-Seq Mapping and Visualization pipeline
                         Tongyi Dou

![ChIP-Seq Overview](https://biosakshat.weebly.com/uploads/1/2/5/1/12512943/chip-seq-overview-dna-bound-protein-fragment-dna-immunoprecipitate_orig.jpg)

In [1]:
#Install Miniconda Python 3 Distribution on my Linux
sh Miniconda3-latest-Linux-x86_64.sh

SyntaxError: invalid syntax (<ipython-input-1-2ec2dc8721c3>, line 2)

In [ ]:
#Install snakemake
conda install -c bioconda -c conda-forge snakemake

In [ ]:
First rule: Mapping

In [ ]:
vi snakefile01

In [ ]:
index = '/home/dell/RefGenome/hg19ebwt/hg19'

rule bowtie_map:
    input:
        fq1 = 'FoxA1.fq',
    output:
        o1 = 'mapped_reads/FoxA1.out',
        o2 = 'mapped_reads/FoxA1.sam',
    shell:
        'bowtie {index} -q {input.fq1} -m 1 -5 10 -3 10 -p 32 -S 2> {output.o1} >{output.o2}'

In [ ]:
snakemake -ns snakefile01

![Enrichment](Step01-Bowtie_Map_01.png)

In [1]:
snakemake -ps snakefile01

SyntaxError: invalid syntax (<ipython-input-1-362c75d7be85>, line 1)

![Enrichment](Step01-Bowtie_Map_02.png)

In [ ]:
Second rule: Sorting

In [ ]:
vi snakefile02

In [ ]:
rule samtools_sort:
    input:
        rules.bowtie_map.output.o2
    output:
        'sorted_reads/FoxA1.bam'
    shell:
        'samtools sort -@ 8 -o {output} {input}'

In [ ]:
snakemake -ns snakefile02

![](Step02-samtools_sort_01.png)

In [ ]:
snakemake -ps snakefile02

![](Step02-samtools_sort_02.png)

In [ ]:
Third rule: Indexing

In [ ]:
vi snakefile03

In [ ]:
rule samtools_index:
    input:
        rules.samtools_sort.output 
    output:
        'sorted_reads/FoxA1.bam.bai'
    shell:
        'samtools index {input} {output}'

In [ ]:
snakemake -ns snakefile03

![](Step03-samtools_index_01.png)

In [ ]:
snakemake -ps snakefile03

![](Step03-samtools_index_02.png)

In [ ]:
Combine rules

In [ ]:
vi snakefile04

In [ ]:
rule all:
    input:
        'sorted_reads/FoxA1.bam.bai'

index = '/home/dell/RefGenome/hg19ebwt/hg19'

rule bowtie_map:
    input:
        fq1 = 'FoxA1.fq',
    output:
        o1 = 'mapped_reads/FoxA1.out',
        o2 = 'mapped_reads/FoxA1.sam',
    shell:
        'bowtie {index} -q {input.fq1} -m 1 -5 10 -3 10 -p 32 -S 2> {output.o1} >{output.o2}'

rule samtools_sort:
    input:
        rules.bowtie_map.output.o2
    output:
        'sorted_reads/FoxA1.bam'
    shell:
        'samtools sort -@ 8 -o {output} {input}'
        
rule samtools_index:
    input:
        rules.samtools_sort.output 
    output:
        'sorted_reads/FoxA1.bam.bai'
    shell:
        'samtools index {input} {output}'

In [ ]:
snakemake -ns snakefile04

![](Step04-AllStep_01.png)

In [ ]:
snakemake -ps snakefile04

![](Step04-AllStep_02.png)
![](Step04-AllStep_03.png)

In [ ]:
Use Wildcards

In [ ]:
vi snakefile05

In [ ]:
rule all:
    input:
        'sorted_reads/{sample}.bam.bai'

index = '/home/dell/RefGenome/hg19ebwt/hg19'

rule bowtie_map:
    input:
        fq1 = '{sample}.fq',
    output:
        o1 = 'mapped_reads/{sample}.out',
        o2 = 'mapped_reads/{sample}.sam',
    shell:
        'bowtie {index} -q {input.fq1} -m 1 -5 10 -3 10 -p 32 -S 2> {output.o1} >{output.o2}'

rule samtools_sort:
    input:
        rules.bowtie_map.output.o2
    output:
        'sorted_reads/{sample}.bam'
    shell:
        'samtools sort -@ 8 -o {output} {input}'

rule samtools_index:
    input:
        rules.samtools_sort.output 
    output:
        'sorted_reads/{sample}.bam.bai'
    shell:
        'samtools index {input} {output}'

In [ ]:
snakemake -ns snakefile sorted_reads/FoxA1.bam.bai sorted_reads/Input.bam.bai sorted_reads/298-301.bam.bai

![](Step05-WildCard_01.png)

In [ ]:
snakemake -ps snakefile sorted_reads/FoxA1.bam.bai sorted_reads/Input.bam.bai sorted_reads/298-301.bam.bai

![](Step05-WildCard_02.png)
![](Step05-WildCard_03.png)

In [ ]:
Add more rules...

In [ ]:
rule all:
    input:
        'sorted_reads/{sample}.bam.bai',
        'sorted_reads/{sample}.bw',
        'read_coverage/{sample}_matrix_TSS.gz',
        'read_coverage/{sample}_regions_genes.bed',
        'peak_visualization/{sample}_Profile.png',
        'peak_visualization/{sample}_Heatmap.png'

index = '/home/dell/RefGenome/hg19ebwt/hg19'
reference = '/home/dell/Downloads/Galaxy1-[UCSC_Main_on_Human__xenoRefGene_(genome)].bed'

rule bowtie_map:
    input:
        fq1 = '{sample}.fq',
    output:
        o1 = 'mapped_reads/{sample}.out',
        o2 = 'mapped_reads/{sample}.sam'
    shell:
        'bowtie {index} -q {input.fq1} -m 1 -5 10 -3 10 -p 32 -S 2> {output.o1} >{output.o2}'

rule samtools_sort:
    input:
        rules.bowtie_map.output.o2
    output:
        'sorted_reads/{sample}.bam'
    shell:
        'samtools sort -@ 8 -o {output} {input}'

rule samtools_index:
    input:
        rules.samtools_sort.output 
    output:
        'sorted_reads/{sample}.bam.bai'
    shell:
        'samtools index {input} {output}'

rule bam2bw:
    input:
        rules.samtools_sort.output 
    output:
        'sorted_reads/{sample}.bw'
    shell:
        'bamCoverage -e 170 -bs 10 -b {input} -o {output}'

rule computeMatrix:
    input:
        rules.bam2bw.output 
    output:
        o1 = 'read_coverage/{sample}_matrix_TSS.gz',
        o2 = 'read_coverage/{sample}_regions_genes.bed'
    shell:
        'computeMatrix reference-point --referencePoint TSS -b 3000 -a 5000 -R {reference} -S {input} --skipZeros -out {output.o1} --outFileSortedRegions {output.o2}'

rule peak_visualization:
    input:
        rules.computeMatrix.output.o1
    output:
        o1 = 'peak_visualization/{sample}_Profile.png',
        o2 = 'peak_visualization/{sample}_Heatmap.png'
    shell:
        'plotProfile -m {input} -out {output.o1} --numPlotsPerRow 2 --plotTitle "Data profile"'
        'plotHeatmap -m {input} -out {output.o2}'

In [ ]:
snakemake -ns snakefile peak_visualization/FoxA1_Profile.png peak_visualization/298-301_Profile.png peak_visualization/Input_Profile.png

![](Step10_01.png)

In [ ]:
Visualization of the DAG using the dot command

In [ ]:
snakemake -ns snakefile peak_visualization/FoxA1_Profile.png peak_visualization/298-301_Profile.png peak_visualization/Input_Profile.png --dag  | dot -Tsvg > dag.svg

![](bag.png)
snakemake -ps snakefile peak_visualization/FoxA1_Profile.png peak_visualization/298-301_Profile.png peak_visualization/Input_Profile.png

In [ ]:
Data Visualization using Deeptools

![Enrichment](Enrichment.png)

In [ ]:
#Data Visualization using the Integrative Genomics Viewer (IGV)
